<a href="https://colab.research.google.com/github/solomontessema/Generative-AI-with-Python/blob/main/notebooks/Vector_Databases_with_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone

In [ ]:
!pip install vec2text

In [ ]:

import pinecone
import openai
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Create Pinecone client
pc = Pinecone(api_key=api_key)

# Create index if it doesn't exist
if "my-index" not in pc.list_indexes().names():
    pc.create_index(
        name="my-index",
        dimension=1536,  # Match your embedding model output
        metric="cosine",  # Or 'euclidean', 'dotproduct'
        spec=ServerlessSpec(
            cloud="aws",  # Or 'gcp'
            region="us-east-1"
        )
    )

# Connect to index
index = pc.Index("my-index")
client = OpenAI(api_key=openai_api_key)

# Embed text using OpenAI
def embed(text):
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

# Upsert vector
index.upsert([
    ("concept1", embed("The theory of relativity and its implications"), {"topic": "physics"}),
    ("concept2", embed("The role of empathy in leadership"), {"topic": "psychology"})
])

# Query
query_vector = embed("My cat is so cute")
results = index.query(vector=query_vector, top_k=5, include_metadata=True)
results


AssertionError: embedder to invert `text-embedding-bge not in list of supported models: ['text-embedding-ada-002', 'gtr-base']`